In [54]:
import tensorflow as tf
from utils import save 
import numpy as np
from models._48_net import calib_48Net
from utils.utilities import batch_selection
import sys
from utils.params import *
import time
from utils.utilities import *
import pickle

In [61]:
batch_size = 256
epochs = 300
lr = [5e-3, 1e-3, 5e-4, 1e-4, 1e-5]
bias_init = [0.0]
reg = [0.0, 1e-4]
_48_calib_batch = batch_selection()

In [62]:
result_list = save.loader('calibrate_db_48_test.txt')
valid_input = np.concatenate([result_list[i][0][np.newaxis] for i in range(len(result_list))])
valid_target = np.asarray([result_list[i][1] for i in range(len(result_list))])

Loaded from D:\programing project\python project\trying stuffpickle/


In [63]:
calibrate_db_48 = save.loader('calibrate_db_48_long.txt')
calibrate_db_48 = calibrate_db_48[:int(len(calibrate_db_48) / batch_size) * batch_size]
inputs_calib_48 = np.zeros((len(calibrate_db_48), 48, 48, 3))
targets_calib_48 = np.zeros((len(calibrate_db_48), 45), dtype=np.int32)

for i in range(len(calibrate_db_48)):
    inputs_calib_48[i,:] = calibrate_db_48[i][0]
    targets_calib_48[i,calibrate_db_48[i][1]] = 1

Loaded from D:\programing project\python project\trying stuffpickle/


In [64]:
bat_len = int(len(inputs_calib_48) / batch_size)
total_iters = bat_len * epochs

In [65]:
def valid_correct_percentage(pred, truth, top_numb=3):
    assert(truth.shape[0] == pred.shape[0])
    top_score = [arr.argsort()[-top_numb:][::-1] for arr in pred]
    score_list = [1 if truth[i] in top_score[i] else 0 for i in range(truth.shape[0])]
    percentage = np.sum(score_list) / truth.shape[0]
    return percentage

In [66]:
input_48 = np.zeros((batch_size, p_net_48, p_net_48, 3))
target_48 = np.zeros((batch_size, 45))
pickle_path = 'saver/calib_48/mean_var/'

In [67]:
for r in reg:
    for b in bias_init:
        for l in lr:
            record_l = l
            correct_pnt_top3 = 0
            correct_pnt_top1 = 0
            max_pnt_top3 = 0
            max_pnt_top1 = 0
            bad_cnt = 0
            max_epo = 0
            
            g = tf.Graph()
            with g.as_default():
                input_48_cal = tf.placeholder(tf.float32, [None,48,48,3])
                target_48_cal = tf.placeholder(tf.float32, [None,45])
                keep_prob = tf.placeholder(tf.float32)
                cal_48_net = calib_48Net(input_48_cal, target_48_cal, keep_prob, lr=l, bias_init=b, reg=r)
            
            sess = tf.InteractiveSession(config=tf.ConfigProto(
                allow_soft_placement=True, log_device_placement=True), graph=g)
            sess.run(tf.global_variables_initializer())
            
            for epo in range(epochs):
                loss = 0
                
                feed_valid = {
                    input_48_cal:valid_input,
                    keep_prob:1.0
                }
                pred_target = cal_48_net.prediction.eval(feed_valid)

                correct_pnt_top3 = valid_correct_percentage(pred_target, valid_target, 3)
                correct_pnt_top1 = valid_correct_percentage(pred_target, valid_target, 1)
                
                if correct_pnt_top1 > max_pnt_top1 and epo != 0:
                    max_pnt_top1 = correct_pnt_top1
                    max_pnt_top3 = correct_pnt_top3
                    bad_cnt = 0
                    max_epo = epo + 1
                    saver = tf.train.Saver()
                    saver.save(sess, p_model_dir + 'calib_48/max/_Net48_b{}_lr{}_reg{}.ckpt'.format(b,record_l,r))
                    with open(pickle_path + '_Net48_MV_b={}_lr={}_reg={}.txt'.format(b, record_l, r), 'wb') as f:
                            pickle.dump(mean_var_list, f)
                            f.close()
                else:
                    bad_cnt += 1
                
                if bad_cnt % 4 == 0:
                    l = l * .98
                    
                mean_var_list = np.zeros((4,64))
                
                if bad_cnt >= 30:
                    break

                for bat in range(bat_len):
                    iters = epo * bat_len + bat
                    input_48, target_48 = _48_calib_batch.next_batch(inputs_calib_48, targets_calib_48, batch_size)

                    feed_train = {
                        input_48_cal:input_48,
                        target_48_cal:target_48,
                        keep_prob:.5
                    }

                    feed_out = {
                        input_48_cal:input_48,
                        target_48_cal:target_48,
                        keep_prob:1.0
                    }
                    loss += cal_48_net.loss.eval(feed_out)
                    single_loss = cal_48_net.loss.eval(feed_out)
                    
                    cal_48_net.train_step.run(feed_train)
                    
                    mean_var_list[0,:] += cal_48_net.mean_conv1.eval(feed_out)
                    mean_var_list[1,:] += cal_48_net.var_conv1.eval(feed_out)
                    mean_var_list[2,:] += cal_48_net.mean_conv2.eval(feed_out)
                    mean_var_list[3,:] += cal_48_net.var_conv2.eval(feed_out)

                    sys.stdout.write('\rIteration: ' + str(iters+1) + '/' + str(total_iters) + \
                                     '  Loss: ' + str(loss/(iters+1))[:7] + '/' + str(single_loss) + \
                                     '  CRT_Top1/Top3: ' + str(correct_pnt_top1*100)[:5] + '% / ' + str(correct_pnt_top3*100)[:5] + '%' + \
                                     '  Max_Top1/Top3: ' + str(max_pnt_top1*100)[:5] + '% / ' + str(max_pnt_top3*100)[:5] + '%' + \
                                     '  Max_Epo: ' + str(max_epo) + '  Bad_cnt: ' + str(bad_cnt))
                mean_var_list = mean_var_list / bat_len

Iteration: 10641/23400  Loss: 0.00102/0.401888  CRT_Top1/Top3: 74.11% / 86.11%  Max_Top1/Top3: 74.11% / 86.11%  Max_Epo: 137  Bad_cnt: 0

KeyboardInterrupt: 

In [43]:
for epo in range(epochs):
    
    feed_valid = {
        input_48_cal:valid_input,
        keep_prob:1.0
    }
    pred_target = cal_48_net.prediction.eval(feed_valid)
    
    correct_pnt_top3 = valid_correct_percentage(pred_target, valid_target, 3)
    correct_pnt_top1 = valid_correct_percentage(pred_target, valid_target, 1)
    
    print('Accuracy_TOP1/TOP3:{}% / {}%'.format(str(correct_pnt_top1*100)[:7], str(correct_pnt_top3*100)[:7]))
    # print(_48_calib_batch.current_index)
    for bat in range(bat_len):
        iters = epo * bat_len + bat
        input_48, target_48 = _48_calib_batch.next_batch(inputs_calib_48, targets_calib_48, batch_size)
        
        feed_train = {
            input_48_cal:input_48,
            target_48_cal:target_48,
            keep_prob:.5
        }
        
        feed_out = {
            input_48_cal:input_48,
            target_48_cal:target_48,
            keep_prob:1.0
        }
        loss = cal_48_net.loss.eval(feed_out)
        cal_48_net.train_step.run(feed_train)
        
        sys.stdout.write('\rIteration: ' + str(iters) + '/' + str(total_iters) + '    Loss: ' + str(loss))

Accuracy_TOP1/TOP3:2.11111% / 5.66666%
Iteration: 77/780    Loss: 2.54377Accuracy_TOP1/TOP3:36.5555% / 60.4444%
Iteration: 155/780    Loss: 1.55908Accuracy_TOP1/TOP3:57.0% / 80.6666%
Iteration: 233/780    Loss: 1.19756Accuracy_TOP1/TOP3:61.5555% / 83.6666%
Iteration: 311/780    Loss: 0.943508Accuracy_TOP1/TOP3:65.0% / 87.4444%
Iteration: 389/780    Loss: 0.767564Accuracy_TOP1/TOP3:69.4444% / 88.6666%
Iteration: 467/780    Loss: 0.687124Accuracy_TOP1/TOP3:72.2222% / 89.6666%
Iteration: 545/780    Loss: 0.65522Accuracy_TOP1/TOP3:71.8888% / 88.6666%
Iteration: 623/780    Loss: 0.461942Accuracy_TOP1/TOP3:71.4444% / 88.7777%
Iteration: 701/780    Loss: 0.505466Accuracy_TOP1/TOP3:73.0% / 89.6666%
Iteration: 779/780    Loss: 0.339889

In [37]:
valid_input.shape

(900, 48, 48, 3)